In [ ]:
#%matplotlib notebook
import matplotlib.pyplot as plt
import time
import re
import matrixprofile as mp
from redistimeseries.client import Client

In [ ]:
redis_host = "localhost"
redis_port = 6380
rts = Client(host=redis_host, port=redis_port)

In [ ]:
from_time=0
to_time=-1
aggregation_type="last"
bucket_size_msec=60000
filters=["SYMSET=ACTIVE"]
print("Querying data...")
start=time.time()
bulk_data=rts.mrange(from_time=from_time,to_time=to_time,aggregation_type=aggregation_type,bucket_size_msec=bucket_size_msec,filters=filters)
print("Querying completed:",int(time.time()-start),"sec.")
print("Splitting data into price and volume...")


In [ ]:
def ProcessPriceTS(d):
    key=str(list(d.keys())[0])
    if not re.match('.*price.*', key):
        return
    rest=list(d.get(key))[1]
    ts,data = map(list,zip(*rest)) # timestamps and data
    def drawMP(windowSize):
        profile=mp.compute(data,windows=windowSize)
        x=mp.discover.regimes(profile, num_regimes=20)
        regimes = x['regimes']
        plt.figure()
        plt.plot(data)
        plt.title("{}\nwindow size: {},{}".format(key,windowSize,regimes))
        for regime in regimes:
            if regime > 0:
                plt.axvline(x=regime)
        regimes
        plt.show()
    for w in range(4,40,1):
        drawMP(w)

max = 76
cur = 1
for d in bulk_data:
    if cur > max:
        break
    ProcessPriceTS(d)
    cur = cur + 1

In [ ]:
cur